### Challenge 7
Se utiliza la base de datos obtenida de PhysioNet "VinDr-SpineXR: A large annotated medical image dataset for spinal lesions detection and classification from radiographs". Esta contiene imágenes de rayos X en formato dicom de 13 enfermedades [1]. Se seleccionaron 50 imagenes de 3 tipos: Disc space narrowing, Foramidal steanosis y Vertebral Collpase.

[1] P. Hieu Huy. “VinDr-SpineXR: A large annotated medical image dataset for spinal lesions detection and classification from radiographs”. PhysioNet. Accedido el 24 de mayo de 2024. [En línea]. Disponible: https://physionet.org/content/vindr-spinexr/1.0.0/

In [37]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pydicom
import os
import cv2

In [38]:
def load_dicom_images(folder_path, target_size=(128, 128)):
    images = []
    print(f"Loading DICOM images from: {folder_path}")
    for subdir, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".dicom"):
                dicom_path = os.path.join(subdir, file)
                try:
                    ds = pydicom.dcmread(dicom_path)
                    image = ds.pixel_array
                    if len(image.shape) == 3:  # Verificar si la imagen tiene 3 canales (color)
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
                    image_resized = cv2.resize(image, target_size)  # Redimensionar la imagen
                    images.append(image_resized)
                except Exception as e:
                    print(f"Error reading DICOM file {dicom_path}: {e}")
    print(f"Loaded {len(images)} images from {folder_path}")
    return np.array(images)


In [39]:
# Rutas de las carpetas con las imágenes DICOM
disc_space_narrowing_path = r'D:\DESK\PATRONUS\chllng_dicom\images_dicom\Disc_space_narrowing'
foramidal_stenosis_path = r'D:\DESK\PATRONUS\chllng_dicom\images_dicom\Foramidal_steanosis'
vertebral_collapse_path = r'D:\DESK\PATRONUS\chllng_dicom\images_dicom\Vertebral_collapse'

# Cargar imágenes y redimensionarlas
images_disc_space_narrowing = load_dicom_images(disc_space_narrowing_path)
images_foramidal_stenosis = load_dicom_images(foramidal_stenosis_path)
images_vertebral_collapse = load_dicom_images(vertebral_collapse_path)

Loading DICOM images from: D:\DESK\PATRONUS\chllng_dicom\images_dicom\Disc_space_narrowing
Loaded 50 images from D:\DESK\PATRONUS\chllng_dicom\images_dicom\Disc_space_narrowing
Loading DICOM images from: D:\DESK\PATRONUS\chllng_dicom\images_dicom\Foramidal_steanosis
Loaded 50 images from D:\DESK\PATRONUS\chllng_dicom\images_dicom\Foramidal_steanosis
Loading DICOM images from: D:\DESK\PATRONUS\chllng_dicom\images_dicom\Vertebral_collapse
Loaded 50 images from D:\DESK\PATRONUS\chllng_dicom\images_dicom\Vertebral_collapse


In [40]:
# Seleccionar 50 imágenes de cada enfermedad
images_disc_space_narrowing = images_disc_space_narrowing[:50]
images_foramidal_stenosis = images_foramidal_stenosis[:50]
images_vertebral_collapse = images_vertebral_collapse[:50]

# Crear las etiquetas
labels_disc_space_narrowing = np.full(50, 0)  # 0 para disc space narrowing
labels_foramidal_stenosis = np.full(50, 1)  # 1 para foramidal stenosis
labels_vertebral_collapse = np.full(50, 2)  # 2 para vertebral collapse

# Combinar todas las imágenes y etiquetas
images = np.concatenate((images_disc_space_narrowing, images_foramidal_stenosis, images_vertebral_collapse), axis=0)
labels = np.concatenate((labels_disc_space_narrowing, labels_foramidal_stenosis, labels_vertebral_collapse), axis=0)

# Añadir una dimensión para el canal
images = images[..., np.newaxis]

# Normalizar las imágenes
images = images / np.max(images)

In [41]:
# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Convertir etiquetas a formato one-hot
y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)


In [61]:
# Crear y compilar la red neuronal de 6 capas
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [62]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 324ms/step - accuracy: 0.3424 - loss: 1.1172 - val_accuracy: 0.2083 - val_loss: 1.0964
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 233ms/step - accuracy: 0.3984 - loss: 1.0901 - val_accuracy: 0.2500 - val_loss: 1.0922
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 230ms/step - accuracy: 0.4154 - loss: 1.0624 - val_accuracy: 0.2083 - val_loss: 1.0860
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 237ms/step - accuracy: 0.3854 - loss: 1.0602 - val_accuracy: 0.3333 - val_loss: 1.0832
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 225ms/step - accuracy: 0.4596 - loss: 1.0381 - val_accuracy: 0.4167 - val_loss: 1.0699
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 230ms/step - accuracy: 0.5417 - loss: 1.0138 - val_accuracy: 0.4583 - val_loss: 1.0662
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 243ms/step - accuracy: 0.5755 - loss: 0.9718 - val_accuracy: 0.5000 - val_loss: 1.0084
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 259ms/step - accuracy: 0.6250 - loss: 0.8953 - val_accuracy: 0.5417 - val_loss: